  - 네이버 개발자 api 신청
  - https://developers.naver.com/products/service-api/search/search.md
  - 예제 : https://developers.naver.com/docs/serviceapi/search/blog/blog.md#python

In [64]:
# 네이버 검색 API 예제 - 블로그 검색
import os
import sys
import urllib.request
import json
import html
import pandas as pd

In [57]:
def getNaverSearch(srcText='뉴스',display=10,start=1):
  client_id = "UZngLtPI8EwAfuzTadat"
  client_secret = "aNw38dR4UQ"
  encText = urllib.parse.quote(srcText)
  url = f"https://openapi.naver.com/v1/search/news?query={encText}&display={display}&start={start}" # JSON 결과
  # url = "https://openapi.naver.com/v1/search/blog.xml?query=" + encText # XML 결과
  request = urllib.request.Request(url)
  request.add_header("X-Naver-Client-Id",client_id)
  request.add_header("X-Naver-Client-Secret",client_secret)
  response = urllib.request.urlopen(request)
  rescode = response.getcode()
  if(rescode==200):
      response_body = response.read()
      return json.loads(response_body.decode('utf-8'))
  else:
      print("Error Code:" + rescode)
      return None

In [58]:
result = getNaverSearch('홍수')

In [68]:
def makeCSV(result):
  title_list,originallink_list,link_lilst,description_list = [],[],[],[]
  for data in result['items']:
    title_list.append(html.unescape( data['title']))
    originallink_list.append(html.unescape(data['originallink']))
    link_lilst.append(html.unescape(data['link']))
    description_list.append(html.unescape(data['description']))
  return pd.DataFrame({'title':title_list,'description':description_list,'originallink':originallink_list,'link':link_lilst,})

In [81]:
df_lists = []
for i in range(1,11):
  df_lists.append(makeCSV(getNaverSearch('홍수',start=1)))

In [ ]:
df_lists[0]